# Linear NCEM Example

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import pytorch_lightning as pl
import torch
import anndata as ad
from geome import transforms 
from geome.adata2data import AnnData2DataByCategory
from utils.datasets import DatasetHartmann
from utils.models.linear_ncem import LinearNCEM
from geome.datamodule import GraphAnnDataModule

/home/sel/mambaforge/envs/gnn/lib/python3.10/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/home/sel/mambaforge/envs/gnn/lib/python3.10/site-packages/spatialdata/__init__.py:9: UserWarning: Geopandas was set to use PyGEOS, changing to shapely 2.0 with:

	geopandas.options.use_pygeos = True

If you intended to use PyGEOS, set the option to False.
  _check_geopandas_using_shapely()


In [3]:
fields = {
    'x':['obs/Cluster_preprocessed','obs/donor','obsm/design_matrix'],
    'y':['X']
}


# preprocess = [
#     lambda x,_: transforms.categorize_obs(x,['donor', 'Cluster_preprocessed', 'point']),
#     lambda x,_: transforms.add_design_matrix(x,'obs/Cluster_preprocessed','obs/donor','design_matrix'),
# ]

from geome.transforms import Categorize, AddDesignMatrix, Compose, AddAdjMatrix

adj_matrix_loc = 'obsp/adj_matrix'

preprocess = Compose(
    [
        Categorize(keys=['donor', 'Cluster_preprocessed', 'point'], axis='obs'),
        AddAdjMatrix(adj_matrix_loc, overwrite=True),
    ]
)



transform = Compose(
    [
        AddDesignMatrix('obs/Cluster_preprocessed','obs/donor', adj_matrix_loc, 'design_matrix', overwrite=True),

    ]
)


# a2d = AnnData2DataByCategory(
#     fields=fields,
#     category=category_to_iterate,
#     preprocess=preprocess,
#     yields_edge_index=True,
# )

category_to_iterate = 'point'


a2d = AnnData2DataByCategory(
    fields=fields,
    category=category_to_iterate,
    preprocess=preprocess,
    transform=transform,
)

#Mibitof
dataset = DatasetHartmann(data_path='./example_data/hartmann/')
adatas = list(dataset.img_celldata.values())

# Merge the list of adatas and convert some string to categories as they should be
adata = ad.concat(adatas)


Loading data from raw files
registering celldata


/home/sel/mambaforge/envs/gnn/lib/python3.10/site-packages/anndata/_core/anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


collecting image-wise celldata
adding graph-level covariates
Loaded 58 images with complete data from 4 patients over 63747 cells with 36 cell features and 8 distinct celltypes.


In [4]:
datas = a2d(adata)
datas

[Data(x=[1338, 88], y=[1338, 36]),
 Data(x=[311, 88], y=[311, 36]),
 Data(x=[768, 88], y=[768, 36]),
 Data(x=[1020, 88], y=[1020, 36]),
 Data(x=[2100, 88], y=[2100, 36]),
 Data(x=[1325, 88], y=[1325, 36]),
 Data(x=[1091, 88], y=[1091, 36]),
 Data(x=[1046, 88], y=[1046, 36]),
 Data(x=[618, 88], y=[618, 36]),
 Data(x=[61, 88], y=[61, 36]),
 Data(x=[1316, 88], y=[1316, 36]),
 Data(x=[1540, 88], y=[1540, 36]),
 Data(x=[1822, 88], y=[1822, 36]),
 Data(x=[863, 88], y=[863, 36]),
 Data(x=[564, 88], y=[564, 36]),
 Data(x=[1023, 88], y=[1023, 36]),
 Data(x=[324, 88], y=[324, 36]),
 Data(x=[287, 88], y=[287, 36]),
 Data(x=[636, 88], y=[636, 36]),
 Data(x=[890, 88], y=[890, 36]),
 Data(x=[1235, 88], y=[1235, 36]),
 Data(x=[1020, 88], y=[1020, 36]),
 Data(x=[1241, 88], y=[1241, 36]),
 Data(x=[1438, 88], y=[1438, 36]),
 Data(x=[1021, 88], y=[1021, 36]),
 Data(x=[1632, 88], y=[1632, 36]),
 Data(x=[780, 88], y=[780, 36]),
 Data(x=[524, 88], y=[524, 36]),
 Data(x=[669, 88], y=[669, 36]),
 Data(x=[241,

In [5]:
num_features = datas[0].x.shape[1]
out_channels = datas[0].y.shape[1]
num_features, out_channels

(88, 36)

In [6]:
dm = GraphAnnDataModule(datas=datas, num_workers = 12, batch_size=12,learning_type='node')
model = LinearNCEM(in_channels=num_features,out_channels=out_channels, lr=0.0001,weight_decay=0.000001)

In [7]:
trainer:pl.Trainer = pl.Trainer(accelerator='gpu' if torch.torch.cuda.is_available() else 'cpu',
                                max_epochs=100)



GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/sel/mambaforge/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(


In [8]:
trainer.fit(model,datamodule=dm)

/home/sel/mambaforge/envs/gnn/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py:50: UserWarning: Using '{self.__class__.__name__}' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn("Using '{self.__class__.__name__}' without a "
Missing logger folder: /home/sel/projects/geome/geome/docs/notebooks/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type            | Params
------------------------------------------------
0 | model_sigma | Linear          | 3.2 K 
1 | model_mu    | Linear          | 3.2 K 
2 | loss_module | GaussianNLLLoss | 0     
------------------------------------------------
6.4 K     Trainable params
0         Non-trainable params
6.4 K     Total params
0.026     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

ImportError: Caught ImportError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/sel/mambaforge/envs/gnn/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/sel/mambaforge/envs/gnn/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
  File "/home/sel/mambaforge/envs/gnn/lib/python3.10/site-packages/torch_geometric/loader/node_loader.py", line 117, in collate_fn
    out = self.node_sampler.sample_from_nodes(input_data)
  File "/home/sel/mambaforge/envs/gnn/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py", line 174, in sample_from_nodes
    return node_sample(inputs, self._sample)
  File "/home/sel/mambaforge/envs/gnn/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py", line 358, in node_sample
    out = sample_fn(seed, seed_time)
  File "/home/sel/mambafo

In [9]:
trainer.test(model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

ImportError: Caught ImportError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/sel/mambaforge/envs/gnn/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/sel/mambaforge/envs/gnn/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
  File "/home/sel/mambaforge/envs/gnn/lib/python3.10/site-packages/torch_geometric/loader/node_loader.py", line 117, in collate_fn
    out = self.node_sampler.sample_from_nodes(input_data)
  File "/home/sel/mambaforge/envs/gnn/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py", line 174, in sample_from_nodes
    return node_sample(inputs, self._sample)
  File "/home/sel/mambaforge/envs/gnn/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py", line 358, in node_sample
    out = sample_fn(seed, seed_time)
  File "/home/sel/mambafo